In [1]:
!apt-get install gcc-4.8 gcc-5 g++-4.8 g++-5
!apt-get install libblas-dev liblapack-dev
!apt-get install libopenblas-dev
!apt-get install gfortran-4.8
!apt-get install gfortran-5
!apt-get install scons
!apt-get install libpng-dev
!apt-get install libboost-all-dev freeglut3-dev
!apt-get install xvfb

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  cpp-4.8 cpp-5 gcc-4.8-base gcc-5-base libasan0 libasan2 libgcc-4.8-dev
  libgcc-5-dev libisl15 libmpx0 libstdc++-4.8-dev libstdc++-5-dev
Suggested packages:
  gcc-4.8-locales gcc-5-locales g++-4.8-multilib gcc-4.8-doc
  libstdc++6-4.8-dbg g++-5-multilib gcc-5-doc libstdc++6-5-dbg
  gcc-4.8-multilib libgcc1-dbg libgomp1-dbg libitm1-dbg libatomic1-dbg
  libasan0-dbg libtsan0-dbg libquadmath0-dbg gcc-5-multilib libasan2-dbg
  liblsan0-dbg libubsan0-dbg libcilkrts5-dbg libmpx0-dbg libstdc++-4.8-doc
  libstdc++-5-doc
The following NEW packages will be installed:
  cpp-4.8 cpp-5 g++-4.8 g++-5 gcc-4.8 gcc-4.8-base gcc-5 gcc-5-base libasan0
  libasan2 libgcc-4.8-dev libgcc-5-dev libisl15 libmpx0 libstdc++-4.8-dev
  libstdc++-5-dev
0 upgraded, 16 newly installed, 0 to remove and 21 not upgraded.
Need to get 58.3 MB of archives.
After this operati

In [2]:
!pip install torch==1.3.0

     |████████████████████████████████| 773.1MB 21kB/s 
ERROR: torchvision 0.7.0+cu101 has requirement torch==1.6.0, but you'll have torch 1.3.0 which is incompatible.
  Found existing installation: torch 1.6.0+cu101
    Uninstalling torch-1.6.0+cu101:
      Successfully uninstalled torch-1.6.0+cu101


In [3]:
!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-4.8 10
!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-5 20
!update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-4.8 10
!update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-5 20

update-alternatives: using /usr/bin/gcc-4.8 to provide /usr/bin/gcc (gcc) in auto mode
update-alternatives: using /usr/bin/gcc-5 to provide /usr/bin/gcc (gcc) in auto mode
update-alternatives: using /usr/bin/g++-4.8 to provide /usr/bin/g++ (g++) in auto mode
update-alternatives: using /usr/bin/g++-5 to provide /usr/bin/g++ (g++) in auto mode


In [6]:
!git clone https://github.com/mszarski/diffsim.git

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (5/5), done.
From https://github.com/mszarski/diffsim
 * branch            HEAD       -> FETCH_HEAD
Updating 8e86fc5..0199bf7
Fast-forward
 arcsim/src/displayreplay.cpp | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [5]:
!chmod +x diffsim/arcsim/dependencies/taucs/configure

In [7]:
%cd /content

/content


In [8]:
%cd diffsim
!update-alternatives --set gcc "/usr/bin/gcc-4.8"
!update-alternatives --set g++ "/usr/bin/g++-4.8"
%cd arcsim/dependencies/
!make 
%cd ../..
!update-alternatives --set gcc "/usr/bin/gcc-5"
!update-alternatives --set g++ "/usr/bin/g++-5"
!make -j 8
%cd pysim
!ln -s ../arcsim/conf ./conf
!ln -s ../arcsim/materials ./materials
!ln -s ../arcsim/meshes ./meshes


/content/diffsim
update-alternatives: using /usr/bin/gcc-4.8 to provide /usr/bin/gcc (gcc) in manual mode
update-alternatives: using /usr/bin/g++-4.8 to provide /usr/bin/g++ (g++) in manual mode
/content/diffsim/arcsim/dependencies
make: Nothing to be done for 'all'.
/content/diffsim
update-alternatives: using /usr/bin/gcc-5 to provide /usr/bin/gcc (gcc) in manual mode
update-alternatives: using /usr/bin/g++-5 to provide /usr/bin/g++ (g++) in manual mode
CXX arcsim/src/displayreplay.cpp
g++ arcsim/src/displayreplay.cpp -MMD -MP -DFAST_MODE -pthread -fPIC -fwrapv -std=c++11 -DNDEBUG -O3 -I/usr/include/python3.6m -INo -ICUDA -Iruntime -Iis -Ifound, -Iusing -ICUDA_HOME='/usr/local/cuda' -I/usr/local/lib/python3.6/dist-packages/torch/include -I/usr/local/lib/python3.6/dist-packages/torch/include/torch/csrc/api/include -I/usr/local/lib/python3.6/dist-packages/torch/include/TH -I/usr/local/lib/python3.6/dist-packages/torch/include/THC -Iarcsim/dependencies/include -Iarcsim/src -DTORCH_API_IN

In [ ]:
##restart runtime here

In [ ]:
%cd diffsim/pysim

[Errno 2] No such file or directory: 'diffsim/pysim'
/content/diffsim/pysim


In [ ]:
import torch
import arcsim
import gc
import time
import json
import sysx
import gc
import os
import numpy as np

from datetime import datetime
now = datetime.now()
timestamp = datetime.now().strftime('%Y-%m-%d_%H:%M:%S')

out_path = '/content/default_out'

if not os.path.exists(out_path):
	os.mkdir(out_path)

with open('conf/rigidcloth/bounce/bounce.json','r') as f:
	config = json.load(f)


def save_config(config, file):
	with open(file,'w') as f:
		json.dump(config, f)

save_config(config, out_path+'/conf.json')

torch.set_num_threads(8)
spf = config['frame_steps']
scalev=1

def reset_sim(sim, epoch):

	if epoch < 20:

		arcsim.init_physics(out_path+'/conf.json', out_path+'/out%d'%epoch,False)
		goal = torch.tensor([0.0000, 0.0000, 0.0000, 0, 0, 0],dtype=torch.float64)
		text_name = out_path+'/out%d'%epoch + "/goal.txt"
		np.savetxt(text_name, goal[3:6], delimiter=',')
	else:
		arcsim.init_physics(out_path+'/conf.json',out_path+'/out',False)

	# arcsim.init_physics(sys.argv[1]+'/conf.json',sys.argv[1]+'/out',False)

	print(sim.obstacles[0].curr_state_mesh.dummy_node.x)


def get_loss(ans, param_g):
	#[0.0000, 0.0000, 0.0000, 0.7500, 0.6954, 0.3159
	vec = torch.tensor([0, 0],dtype=torch.float64)
	loss = torch.norm(ans.narrow(0, 3, 2) - vec, p=2)
	reg  = torch.norm(param_g, p=2)*0.001
 
	print(ans)
	print(loss)
	print(reg)

	return loss + reg

def run_sim(steps,sim,param_g):
	
			
	for obstacle in sim.obstacles:
		for node in obstacle.curr_state_mesh.nodes:
			node.m    *= 0.01

	# sim.obstacles[2].curr_state_mesh.dummy_node.x = param_g[1]
	print("step")
	for step in range(20):
		print(step)

		dx = []
		dx.append(torch.zeros([3],dtype=torch.float64))
		dx.append(param_g[step])
		dx.append(torch.zeros([1],dtype=torch.float64))
		dx = torch.cat(dx)
		sim.obstacles[0].curr_state_mesh.dummy_node.v += dx

		arcsim.sim_step()

	cnt = 0

	ans = sim.obstacles[0].curr_state_mesh.dummy_node.x
	ans = ans

	# ans = torch.tensor([0, 0, 0],dtype=torch.float64)
	# for node in sim.cloths[0].mesh.nodes:
	# 	cnt += 1
	# 	ans = ans + node.x
	# ans /= cnt
	loss = get_loss(ans, param_g)

	return loss, ans

def do_train(cur_step,optimizer,sim,param_g):
	epoch = 0
	while True:
		steps=10
		reset_sim(sim, epoch)
		st = time.time()
		loss, ans = run_sim(steps, sim, param_g)
		en0 = time.time()
		optimizer.zero_grad()

	
		loss.backward(retain_graph=True)



		# if loss<8e-2:
		# 	break
		# dgrad, stgrad, begrad = torch.autograd.grad(loss, [density, stretch, bend])
		en1 = time.time()
		# print(ans)
		# asdf.asdf
		print("=======================================")
		#print(param_g.data)
		#print(param_g.grad.data)
		f.write('epoch {}:  loss={} \n'.format(epoch,  loss.data))
		print('epoch {}:  loss={} \n'.format(epoch, loss.data))

		print('forward time={}'.format(en0-st))
		print('backward time={}'.format(en1-en0))


		optimizer.step()
		if epoch>=100:
			break
		epoch = epoch + 1
		# break

with open(out_path+('/log%s.txt'%timestamp),'w',buffering=1) as f:
	tot_step = 1
	sim=arcsim.get_sim()
	# reset_sim(sim)

	param_g = torch.zeros([20, 2],dtype=torch.float64, requires_grad=True)

	lr = 0.1
	momentum = 0.4
	f.write('lr={} momentum={}\n'.format(lr,momentum))
	optimizer = torch.optim.SGD([{'params':param_g,'lr':lr}],momentum=momentum)
	# optimizer = torch.optim.Adadelta([density, stretch, bend])
	for cur_step in range(tot_step):
		do_train(cur_step,optimizer,sim,param_g)

print("done")


tensor([ 0.0000,  0.0000,  0.0000, -0.4794,  0.4905,  0.1025],
       dtype=torch.float64)
step
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
tensor([ 1.6901e-03,  7.6942e-03,  8.0063e-06, -5.7765e-01,  5.1540e-01,
        -5.2140e-02], dtype=torch.float64, grad_fn=<SliceBackward>)
tensor(0.7742, dtype=torch.float64, grad_fn=<NormBackward0>)
tensor(0., dtype=torch.float64, grad_fn=<MulBackward0>)
epoch 0:  loss=0.7741560801143086 

forward time=8.201165199279785
backward time=9.001873970031738
tensor([ 0.0000,  0.0000,  0.0000, -0.4794,  0.4905,  0.1025],
       dtype=torch.float64)
step
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
tensor([-0.0006,  0.0144, -0.0009, -0.3973,  0.3488, -0.0488],
       dtype=torch.float64, grad_fn=<SliceBackward>)
tensor(0.5287, dtype=torch.float64, grad_fn=<NormBackward0>)
tensor(0.0001, dtype=torch.float64, grad_fn=<MulBackward0>)
epoch 1:  loss=0.528855421150579 

forward time=8.968272686004639
backward time=9.136162519454956
tensor([ 0.0000,

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import arcsim
import gc
import time
import json
import sys
import gc
import numpy as np
import os
from datetime import datetime

handles = [25, 60, 30, 54]

out_path = '/content/default_out'

if not os.path.exists(out_path):
	os.mkdir(out_path)


timestamp = datetime.now().strftime('%Y-%m-%d_%H:%M:%S')

torch_model_path = out_path + ('/net_weight.pth%s'%timestamp)


class Net(nn.Module):
	def __init__(self, n_input, n_output):
		super(Net, self).__init__()
		self.fc1 = nn.Linear(n_input, 50).double()
		self.fc2 = nn.Linear(50, 200).double()
		self.fc3 = nn.Linear(200, n_output).double()
	def forward(self, x):
		x = F.relu(self.fc1(x))
		x = F.relu(self.fc2(x))
		x = self.fc3(x)
		# x = torch.clamp(x, min=-5, max=5)
		return x

with open('conf/rigidcloth/drag/drag.json','r') as f:
	config = json.load(f)
# matfile = config['cloths'][0]['materials'][0]['data']
# with open(matfile,'r') as f:
# 	matconfig = json.load(f)

def save_config(config, file):
	with open(file,'w') as f:
		json.dump(config, f)

save_config(config, out_path+'/conf.json')


torch.set_num_threads(8)
spf = config['frame_steps']

scalev=1

def reset_sim(sim, epoch, goal):
	if epoch % 5==0:
		arcsim.init_physics(out_path+'/conf.json', out_path+'/out%d'%epoch,False)
		text_name = out_path+'/out%d'%epoch + "/goal.txt"
		np.savetxt(text_name, goal[3:6], delimiter=',')
	else:
		arcsim.init_physics(out_path+'/conf.json',out_path+'/out',False)
	print(sim.obstacles[0].curr_state_mesh.dummy_node.x)


# def get_loss(ans, goal):
# 	#[0.0000, 0.0000, 0.0000, 0.7500, 0.6954, 0.3159
# 	dif = ans - goal
# 	loss = torch.norm(dif.narrow(0, 3, 3), p=2)

# 	return loss
def get_loss(ans, goal):
	#[0.0000, 0.0000, 0.0000, 0.7500, 0.6954, 0.3159
	diff = ans - goal
	loss = torch.norm(diff.narrow(0, 3, 3), p=2)

	print(ans)
	print(goal)
	print(loss)

	return loss

def run_sim(steps, sim, net, goal):

	for obstacle in sim.obstacles:
		for node in obstacle.curr_state_mesh.nodes:
			node.m    *= 0.2

	#sim.obstacles[0].curr_state_mesh.dummy_node.x = torch.tensor([0.0000, 0.0000, 0.0000,
	#np.random.random(), np.random.random(), -np.random.random()],dtype=torch.float64)
	print(sim.obstacles[0].curr_state_mesh.dummy_node.x)
	for step in range(steps):
		print(step)
		remain_time = torch.tensor([(steps - step)/steps],dtype=torch.float64)
		
		net_input = []
		for i in range(len(handles)):
			net_input.append(sim.cloths[0].mesh.nodes[handles[i]].x)
			net_input.append(sim.cloths[0].mesh.nodes[handles[i]].v)

		# dis = sim.obstacles[0].curr_state_mesh.dummy_node.x - goal
		# net_input.append(dis.narrow(0, 3, 3))
		net_input.append(remain_time)
		net_output = net(torch.cat(net_input))
		

		# outputs = net_output.view([4, 3])
		
		for i in range(len(handles)):
			sim_input = torch.cat([torch.tensor([0, 0],dtype=torch.float64), net_output[i].view([1])])
			sim.cloths[0].mesh.nodes[handles[i]].v += sim_input 

		arcsim.sim_step()

	cnt = 0
	ans1 = torch.tensor([0, 0, 0],dtype=torch.float64)
	for node in sim.cloths[0].mesh.nodes:
		cnt += 1
		ans1 = ans1 + node.x
	ans1 /= cnt

	ans1 = torch.cat([torch.tensor([0, 0, 0],dtype=torch.float64),
							ans1])

	# ans  = ans1
	ans = sim.obstacles[0].curr_state_mesh.dummy_node.x
	
	loss = get_loss(ans1, goal)

	return loss, ans

def do_train(cur_step,optimizer,sim,net):
	epoch = 0
	while True:
		# steps = int(1*15*spf)
		steps = 20

		sigma = 0.05
		z = np.random.random()*sigma + 0.5

		y = np.random.random()*sigma - sigma/2
		x = np.random.random()*sigma - sigma/2


		ini_co = torch.tensor([0.0000, 0.0000, 0.0000,0.4744, 0.4751, 0.0564], dtype=torch.float64)
		goal = torch.tensor([0.0000, 0.0000, 0.0000,
		 0, 0, z],dtype=torch.float64)
		goal = goal + ini_co

		reset_sim(sim, epoch, goal)

		st = time.time()
		loss, ans = run_sim(steps, sim, net, goal)
		en0 = time.time()
		
		optimizer.zero_grad()

		loss.backward()

		en1 = time.time()
		print("=======================================")
		f.write('epoch {}: loss={}\n  ans = {}\n goal = {}\n'.format(epoch, loss.data,  ans.narrow(0,3,3).data, goal.data))
		print('epoch {}: loss={}\n  ans = {}\n goal = {}\n'.format(epoch, loss.data,  ans.narrow(0,3,3).data, goal.data))
		#print('epoch {}: loss={}  ans={}\n'.format(epoch, loss.data, ans.data))

		print('forward tim = {}'.format(en0-st))
		print('backward time = {}'.format(en1-en0))


		if epoch % 5 == 0:
			torch.save(net.state_dict(), torch_model_path)

		if loss<1e-3:
			break
		# dgrad, stgrad, begrad = torch.autograd.grad(loss, [density, stretch, bend])

		optimizer.step()
		if epoch>=400:
			quit()
		epoch = epoch + 1
		# break

with open(out_path+('/log%s.txt'%timestamp),'w',buffering=1) as f:
	tot_step = 1
	sim=arcsim.get_sim()
	# reset_sim(sim)

	#param_g = torch.tensor([0,0,0,0,0,1],dtype=torch.float64, requires_grad=True)
	net = Net(25, 4)
	if os.path.exists(torch_model_path):
		net.load_state_dict(torch.load(torch_model_path))
		print("load: %s\n success" % torch_model_path)

	lr = 0.01
	momentum = 0.9
	f.write('lr={} momentum={}\n'.format(lr,momentum))
	#optimizer = torch.optim.SGD([{'params':net.parameters(),'lr':lr}],momentum=momentum)
	optimizer = torch.optim.Adam(net.parameters(),lr=lr)
	# optimizer = torch.optim.Adadelta([density, stretch, bend])
	for cur_step in range(tot_step):
		do_train(cur_step,optimizer,sim,net)

print("done")


In [1]:
%cd /content/diffsim/pysim

/content/diffsim/pysim


In [2]:
!python exp_inverse.py

['exp_inverse.py']
tensor([ 0.0000,  0.0000,  0.0000, -0.4794,  0.4905,  0.1025],
       dtype=torch.float64)
step
0
1
2
3
4
5
6
Traceback (most recent call last):
  File "exp_inverse.py", line 152, in <module>
    do_train(cur_step,optimizer,sim,param_g)
  File "exp_inverse.py", line 108, in do_train
    loss, ans = run_sim(steps, sim, param_g)
  File "exp_inverse.py", line 86, in run_sim
    arcsim.sim_step()
  File "/content/diffsim/pysim/taucs_py.py", line 6, in forward
    @staticmethod
KeyboardInterrupt


In [2]:
%%writefile visualize.py
import torch
import arcsim

with torch.autograd.profiler.profile() as prof:
	  arcsim.msim(4,['arcsim','replay','default_out/out0', 'default_out/out0'])
print(prof)

Overwriting visualize.py


In [3]:
!xvfb-run -s "-screen 0 1024x768x24 -ac +extension GLX +render -noreset" python visualize.py

main
4
arcsim replay default_out/out0
reply 1 
reply 2 
reply 3 
reply 4 
reply 5 
reply 6 
default_out/out0/0000_
default_out/out0/0000_rig
reply 7 
default_out/out0/0000.bmp
default_out/out0/0001_
default_out/out0/0001_rig
default_out/out0/0001.bmp
default_out/out0/0002_
default_out/out0/0002_rig
default_out/out0/0002.bmp
default_out/out0/0003_
default_out/out0/0003_rig
default_out/out0/0003.bmp
default_out/out0/0004_
default_out/out0/0004_rig
default_out/out0/0004.bmp
default_out/out0/0005_
default_out/out0/0005_rig
default_out/out0/0005.bmp
default_out/out0/0006_
default_out/out0/0006_rig
default_out/out0/0006.bmp
default_out/out0/0007_
default_out/out0/0007_rig
default_out/out0/0007.bmp
default_out/out0/0008_
default_out/out0/0008_rig
default_out/out0/0008.bmp
default_out/out0/0009_
default_out/out0/0009_rig
default_out/out0/0009.bmp
default_out/out0/0010_
default_out/out0/0010_rig
default_out/out0/0010.bmp
default_out/out0/0011_
default_out/out0/0011_rig
default_out/out0/0011.bmp

In [19]:
import pathlib
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

frames = []
for bmp_file in sorted(list(pathlib.Path('default_out/out0').glob('*.bmp'))):
    frames.append(plt.imread(bmp_file))
    print(bmp_file)
    

default_out/out0/0000.bmp
default_out/out0/0001.bmp
default_out/out0/0002.bmp
default_out/out0/0003.bmp
default_out/out0/0004.bmp
default_out/out0/0005.bmp
default_out/out0/0006.bmp
default_out/out0/0007.bmp
default_out/out0/0008.bmp
default_out/out0/0009.bmp
default_out/out0/0010.bmp
default_out/out0/0011.bmp
default_out/out0/0012.bmp
default_out/out0/0013.bmp
default_out/out0/0014.bmp
default_out/out0/0015.bmp
default_out/out0/0016.bmp
default_out/out0/0017.bmp
default_out/out0/0018.bmp
default_out/out0/0019.bmp
default_out/out0/0020.bmp
default_out/out0/0021.bmp
default_out/out0/0022.bmp
default_out/out0/0023.bmp
default_out/out0/0024.bmp
default_out/out0/0025.bmp
default_out/out0/0026.bmp
default_out/out0/0027.bmp
default_out/out0/0028.bmp
default_out/out0/0029.bmp
default_out/out0/0030.bmp
default_out/out0/0031.bmp
default_out/out0/0032.bmp
default_out/out0/0033.bmp
default_out/out0/0034.bmp
default_out/out0/0035.bmp
default_out/out0/0036.bmp
default_out/out0/0037.bmp
default_out/

In [ ]:
from matplotlib import animation, rc
from IPython.display import HTML

fig = plt.figure()

ims = []
for i in range(len(frames)):
    im = plt.imshow(frames[i], animated=True)
    ims.append([im])

ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True,
                                repeat_delay=1000)

rc('animation', html='jshtml')
ani